This notebook contains the experiments for the "Evaluating probabilistic forecasters with sktime - an easy-to-use, highly configurable framework for reproducible science"
paper.



## Used Forecasters:
* StatsForecastAutoArima
* StatsForecastAutoTheta
* NaiveForecaster


## Used Wrappers
* ConformalIntervals with different methods
* tsbootstrap
* SquaringResiduals


## Used Metrics
* CRPS
* PinballLoss
* EmpricalCoverage
* Sharpness
* Calibration (AUCalibration)

## Used Datasets
* australian_electricity_demand_dataset
* sunspot dataset
* us-birth dataset



In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sktime.benchmarking.forecasting import ForecastingBenchmark
from sktime.datasets import load_forecastingdata
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting.probabilistic import CRPS, PinballLoss, EmpiricalCoverage, AUCalibration, IntervalWidth
from sktime.split import SlidingWindowSplitter
from sktime.forecasting.statsforecast import  StatsForecastAutoTheta, StatsForecastAutoARIMA
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.compose import BaggingForecaster

from sktime.transformations.bootstrap import TSBootstrapAdapter

from tsbootstrap import (MovingBlockBootstrap,     BlockDistributionBootstrap,
    BlockResidualBootstrap,
    BlockStatisticPreservingBootstrap,)
from sktime.forecasting.conformal import ConformalIntervals
from sktime.forecasting.squaring_residuals import SquaringResiduals

from sktime.forecasting.naive import NaiveForecaster, NaiveVariance



## Define some utility functions

In [8]:
def create_forecasting_pipeline(forecaster, sp=12):
    return Differencer(lags=1) * Deseasonalizer(sp) * forecaster


def get_model_name(x):
    if x.startswith("naive"):
        return x.split("_")[0] + "_" + x.split("_")[1]
    else:
        return x.split("_")[0]

def get_wrapper_id(x):
    if x.startswith("naive"):
        id_ =  "_".join(x.split("_")[2:])
    else:
        id_ = "_".join(x.split("_")[1:])
    
    return {
        "BaggingForecaster_BlockDistributionBootstrap" : "BlockDistributionBootstrap",
        "BaggingForecaster_BlockResidualBootstrap" : "BlockResidualBootstrap",
        "BaggingForecaster_BlockStatisticPreservingBootstrap" : "BlockStatisticPreservingBootstrap",
        "BaggingForecaster_MovingBlockBootstrap" : "MovingBlockBootstrap",
        "Fallback" : "Fallback",
        "ConformalIntervals_empirical" : "CI Empirical",
        "ConformalIntervals_empirical_residual" : "CI Empirical Residual",
        "ConformalIntervals_M" : "CI Bonferroni ",
        "ConformalIntervals_conformal_bonferroni" : "CI Bonferroni ",
        "ConformalIntervals_conformal" : "CI Conformal",
        "NaiveVariance" : "Naive Variance",
        "SquaringResiduals" : "Squaring Residuals",

    }[id_]


def get_ds_id(x):
    if "_t1" in x:
        return "T1"
    elif "_t2" in x:
        return "T2"
    elif "_t3" in x:
        return "T3"
    elif "_t4" in x:
        return "T4"
    elif "_t5" in x:
        return "T5"
    
plt.rc('font', size=8.0)


## Define the data loaders

In [3]:


def sunspot():
    return load_forecastingdata("sunspot_dataset_without_missing_values", return_type="pd_multiindex_hier")[0].loc["T1"][-365*40:]

def us_birth():
    return load_forecastingdata("us_births_dataset", return_type="pd_multiindex_hier")[0]


def australian_electricity_demand_dataset_t1(series="T1"):
    data = load_forecastingdata("australian_electricity_demand_dataset")[0]
    data = data.set_index("series_name")
    series = data.loc[series]["series_value"]
    return pd.DataFrame(series, index=pd.date_range("2006-01-01", periods=len(series), freq="30min"))[-365*48:]

def australian_electricity_demand_dataset_t2(series="T2"):
    data = load_forecastingdata("australian_electricity_demand_dataset")[0]
    data = data.set_index("series_name")
    series = data.loc[series]["series_value"]
    return pd.DataFrame(series, index=pd.date_range("2006-01-01", periods=len(series), freq="30min"))[-365*48:]

def australian_electricity_demand_dataset_t3(series="T3"):
    data = load_forecastingdata("australian_electricity_demand_dataset")[0]
    data = data.set_index("series_name")
    series = data.loc[series]["series_value"]
    return pd.DataFrame(series, index=pd.date_range("2006-01-01", periods=len(series), freq="30min"))[-365*48:]

def australian_electricity_demand_dataset_t4(series="T4"):
    data = load_forecastingdata("australian_electricity_demand_dataset")[0]
    data = data.set_index("series_name")
    series = data.loc[series]["series_value"]
    return pd.DataFrame(series, index=pd.date_range("2006-01-01", periods=len(series), freq="30min"))[-365*48:]

def australian_electricity_demand_dataset_t5(series="T5"):
    data = load_forecastingdata("australian_electricity_demand_dataset")[0]
    data = data.set_index("series_name")
    series = data.loc[series]["series_value"]
    return pd.DataFrame(series, index=pd.date_range("2006-01-01", periods=len(series), freq="30min"))[-365*48:]



## Function for creating the benchmarking

In [4]:
def create_benchmarking(sp=48, benchmark=None, sample_frac=0.005):
    if benchmark is None:
        benchmark = ForecastingBenchmark()

    for forecaster_name, forecaster in [("naive_last", NaiveForecaster(strategy="last", sp=sp)),
                        ("naive_mean", NaiveForecaster(strategy="mean", sp=sp)),
                        ("naive_dirft", NaiveForecaster(strategy="drift", sp=sp)),
                        ("StatsForecastAutoARIMA", StatsForecastAutoARIMA(sp=sp, n_fits=20, approximation=True)), 
                        ("StatsForecastAutoTheta", StatsForecastAutoTheta(season_length=sp)), 
                       ]:
        for wrapper_name, wrapper in [("ConformalIntervals_empirical", ConformalIntervals(forecaster, sample_frac=sample_frac)),
                                      ("ConformalIntervals_empirical_residual", ConformalIntervals(forecaster, sample_frac=sample_frac, method="empirical_residual")),
                                      ("ConformalIntervals_M", ConformalIntervals(forecaster, sample_frac=sample_frac, method="conformal_bonferroni")),
                                      ("ConformalIntervals_conformal", ConformalIntervals(forecaster, sample_frac=sample_frac, method="conformal")),
                                      ("SquaringResiduals", SquaringResiduals(forecaster, initial_window=14*sp)),
                                      ("NaiveVariance", NaiveVariance(forecaster, initial_window=14*sp)),
                                      ("Fallback", forecaster),
                                      ("BaggingForecaster_MovingBlockBootstrap", BaggingForecaster(TSBootstrapAdapter(MovingBlockBootstrap()), forecaster)),
                                      ("BaggingForecaster_BlockDistributionBootstrap", BaggingForecaster(TSBootstrapAdapter(BlockDistributionBootstrap()), forecaster)),
                                      ("BaggingForecaster_BlockResidualBootstrap", BaggingForecaster(TSBootstrapAdapter(BlockResidualBootstrap()), forecaster)),
                                      ("BaggingForecaster_BlockStatisticPreservingBootstrap", BaggingForecaster(TSBootstrapAdapter(BlockStatisticPreservingBootstrap()), forecaster)),
                                      ]:
            benchmark.add_estimator(
                estimator=create_forecasting_pipeline(wrapper, sp=sp),
                estimator_id=forecaster_name + "_" + wrapper_name,
            )                  
    return benchmark


# Benchmark on australian Electrical time series
* 5 time series, we took only the last year for our benchmarking
* We trained on about one month (48*30 time steps) to predict the next day.
* The step length for the benchmark was also about one month (48*30 time steps)
* Leading to twelve folds.

In [5]:
dataset_loaders = [
    australian_electricity_demand_dataset_t1,
    australian_electricity_demand_dataset_t2,    
    australian_electricity_demand_dataset_t3,    
    australian_electricity_demand_dataset_t4,    
    australian_electricity_demand_dataset_t5,    
    ]


for dataset_loader in dataset_loaders:

    cv_splitter = SlidingWindowSplitter(
        step_length=48*30,
        window_length=48*30,
        fh=range(48)
    )
    scorers = [CRPS(), PinballLoss(), EmpiricalCoverage(), AUCalibration(), IntervalWidth()]

    benchmark = create_benchmarking(sp=48)

    benchmark.add_task(
            dataset_loader,
            cv_splitter,
            scorers,
        )
    results_df = benchmark.run("./australian_electricity.csv",
                                artefacts_store_dir="./artefacts/electricity",
                                )
    results_df.T


/Users/benediktheidrich/code/SciPy_conf_paper/venv/lib/python3.11/site-packages/statsforecast/core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
results_df

,validation_id,model_id,runtime_secs,CRPS_fold_0_test,CRPS_fold_1_test,CRPS_fold_2_test,CRPS_fold_3_test,CRPS_fold_4_test,CRPS_fold_5_test,CRPS_fold_6_test,...,IntervalWidth_fold_4_test,IntervalWidth_fold_5_test,IntervalWidth_fold_6_test,IntervalWidth_fold_7_test,IntervalWidth_fold_8_test,IntervalWidth_fold_9_test,IntervalWidth_fold_10_test,IntervalWidth_fold_11_test,IntervalWidth_mean,IntervalWidth_std
0,[dataset=australian_electricity_demand_dataset...,StatsForecastAutoARIMA_BaggingForecaster_Block...,2268.077026,539.399852,103.752836,80.555662,214.353176,170.200532,95.797210,457.990247,...,995.958883,898.897385,772.556083,914.719185,545.800374,637.270031,580.092597,723.647467,778.854191,152.190597
1,[dataset=australian_electricity_demand_dataset...,StatsForecastAutoARIMA_BaggingForecaster_Block...,2151.175026,460.500332,92.925892,78.354877,203.814670,170.715928,76.315146,501.938675,...,676.235935,788.756972,794.560941,867.863671,389.249618,302.764872,808.063115,491.397960,671.498515,199.035703
2,[dataset=australian_electricity_demand_dataset...,StatsForecastAutoARIMA_BaggingForecaster_Block...,827.127333,469.617494,90.446999,101.812879,228.791155,191.805231,70.457351,572.237164,...,327.924671,297.026772,347.033832,428.448073,261.229678,187.656811,296.464675,234.258965,358.242116,146.231878
3,[dataset=australian_electricity_demand_dataset...,StatsForecastAutoARIMA_BaggingForecaster_Movin...,808.974646,399.163096,96.571716,78.534087,217.280000,159.750426,75.708243,604.911317,...,442.787753,1418.771213,385.589771,329.380876,220.829479,260.325642,546.404289,243.289259,456.260545,322.358091
4,[dataset=australian_electricity_demand_dataset...,StatsForecastAutoARIMA_ConformalIntervals_conf...,1973.664798,430.290662,242.130287,246.719468,384.354853,401.682345,278.899535,378.889961,...,5146.015257,3967.531139,4082.494077,4204.524782,5937.734363,4268.983980,3441.397617,4791.752737,4381.135097,761.944865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,[dataset=australian_electricity_demand_dataset...,naive_mean_ConformalIntervals_empirical,0.590107,45.828770,47.261020,53.685107,46.624560,50.286858,42.273486,58.485256,...,636.093809,611.913000,703.299175,628.969883,343.099328,413.279960,415.771030,379.939695,549.621754,127.228362
229,[dataset=australian_electricity_demand_dataset...,naive_mean_ConformalIntervals_empirical_residual,0.599643,31.956807,35.816472,28.172150,42.087569,38.679357,34.143544,52.819164,...,72.323842,74.606012,78.202099,63.070853,66.978680,66.600345,58.189932,54.042219,69.664269,13.078784
230,[dataset=australian_electricity_demand_dataset...,naive_mean_Fallback,0.404367,56.861120,67.800109,66.264262,76.519899,70.970965,76.988893,77.503562,...,922.286784,1062.688050,922.913579,956.863105,763.261261,651.607534,538.615287,650.204119,835.287281,163.301974
231,[dataset=australian_electricity_demand_dataset...,naive_mean_NaiveVariance,11.356147,62.023710,71.881700,67.531209,78.273558,74.654633,80.994854,80.215804,...,978.523398,1120.208981,968.312268,1089.527144,713.779855,666.010116,573.681896,684.726461,877.785944,181.668791


# Benchmark on Sunspot and USBirt time series
* We trained on one year of data (365 time steps)
* We predict about one month 30 days
* The step length was 395 days -> around 13 months so that not always the same months are predicted..

In [7]:
cv_splitter = SlidingWindowSplitter(
    step_length=395,
    window_length=365,
    fh=range(28)
)
scorers = [CRPS(), PinballLoss(), EmpiricalCoverage(), AUCalibration(), IntervalWidth()]

benchmark = create_benchmarking(sp=1, sample_frac=0.1)
dataset_loaders = [
    us_birth,
    ]

for dataset_loader in dataset_loaders:
    benchmark.add_task(
        dataset_loader,
        cv_splitter,
        scorers,
    )
results_df = benchmark.run("./us_birth.csv",
                           artefacts_store_dir="./artefacts/us_birth"
                           )
results_df.T


,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
validation_id,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...,[dataset=us_birth]_[cv_splitter=SlidingWindowS...
model_id,StatsForecastAutoTheta_BaggingForecaster_Block...,StatsForecastAutoTheta_BaggingForecaster_Block...,StatsForecastAutoTheta_BaggingForecaster_Block...,StatsForecastAutoTheta_BaggingForecaster_Movin...,StatsForecastAutoTheta_ConformalIntervals_M,StatsForecastAutoTheta_ConformalIntervals_conf...,StatsForecastAutoTheta_ConformalIntervals_empi...,StatsForecastAutoTheta_ConformalIntervals_empi...,StatsForecastAutoTheta_Fallback,StatsForecastAutoTheta_NaiveVariance,...,naive_mean_BaggingForecaster_BlockResidualBoot...,naive_mean_BaggingForecaster_BlockStatisticPre...,naive_mean_BaggingForecaster_MovingBlockBootstrap,naive_mean_ConformalIntervals_M,naive_mean_ConformalIntervals_conformal,naive_mean_ConformalIntervals_empirical,naive_mean_ConformalIntervals_empirical_residual,naive_mean_Fallback,naive_mean_NaiveVariance,naive_mean_SquaringResiduals
runtime_secs,4.844241,4.356884,4.495765,4.503388,6.96474,7.108428,7.26881,7.141899,0.790073,75.631698,...,1.518254,1.448176,1.436835,1.079544,1.095921,1.106592,1.080906,0.461852,6.693555,31.855921
CRPS_fold_0_test,1699.519469,4855.240677,1647.423907,2428.333586,8812.703523,3494.466598,3293.150983,2703.237332,3637.515357,3513.020197,...,2844.995178,2541.910683,2631.82041,8734.056127,3181.10665,3029.143971,2217.057039,3131.919466,3155.37634,3454.274489
CRPS_fold_1_test,585.168016,1509.756158,1201.935116,703.946655,9407.803796,2826.332086,2638.868404,1495.546148,3109.50494,2978.439853,...,447.641164,458.038641,416.202041,9008.309239,2741.225687,2695.400399,1528.567958,2901.95628,2938.895327,2302.418245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IntervalWidth_fold_15_test,7919.842533,9345.241715,10510.761017,13652.871864,73804.394341,64790.696057,57000.539952,2505.136706,62773.879415,59226.279488,...,1326.589884,6.941736,1298.17468,70875.546772,64905.345707,62952.068644,1959.855372,58190.24454,58549.092887,43467.786072
IntervalWidth_fold_16_test,10019.374191,5904.483433,8110.074181,11195.861552,80724.266544,66927.125264,60197.964803,2377.788763,65816.790874,61577.986964,...,768.694641,0.203304,1484.048246,77072.579706,65252.933606,56412.291281,2057.771408,60768.459037,60851.443428,45163.848977
IntervalWidth_fold_17_test,10858.542709,3009.703897,5347.550095,5461.612847,80340.167008,68689.842026,64435.477051,1609.243934,66981.925528,62491.956142,...,1265.919723,0.276757,1183.702311,79408.064647,67345.880806,65973.991948,1980.559221,62206.057479,61962.737589,40297.401443
IntervalWidth_mean,8185.700665,4642.530206,5437.895289,5640.290515,59060.936262,47750.909641,45062.677755,1814.012977,48079.185385,45088.580966,...,816.846752,1.450712,857.565248,58143.983293,47318.051673,44337.653546,1725.002033,44493.376415,44601.580957,32888.550884


In [8]:
cv_splitter = SlidingWindowSplitter(
    step_length=395,
    window_length=365,
    fh=range(28)
)
scorers = [CRPS(), PinballLoss(), EmpiricalCoverage(), AUCalibration(), IntervalWidth()]

benchmark = create_benchmarking(sp=1, sample_frac=0.1)
dataset_loaders = [
    sunspot,
    ]

for dataset_loader in dataset_loaders:
    benchmark.add_task(
        dataset_loader,
        cv_splitter,
        scorers,
    )
results_df = benchmark.run("./sunspot.csv",
                           artefacts_store_dir="./artefacts/sunspot"
                           )
results_df.T


,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
validation_id,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...,[dataset=sunspot]_[cv_splitter=SlidingWindowSp...
model_id,StatsForecastAutoTheta_BaggingForecaster_Block...,StatsForecastAutoTheta_BaggingForecaster_Block...,StatsForecastAutoTheta_BaggingForecaster_Block...,StatsForecastAutoTheta_BaggingForecaster_Movin...,StatsForecastAutoTheta_ConformalIntervals_M,StatsForecastAutoTheta_ConformalIntervals_conf...,StatsForecastAutoTheta_ConformalIntervals_empi...,StatsForecastAutoTheta_ConformalIntervals_empi...,StatsForecastAutoTheta_Fallback,StatsForecastAutoTheta_NaiveVariance,...,naive_mean_BaggingForecaster_BlockResidualBoot...,naive_mean_BaggingForecaster_BlockStatisticPre...,naive_mean_BaggingForecaster_MovingBlockBootstrap,naive_mean_ConformalIntervals_M,naive_mean_ConformalIntervals_conformal,naive_mean_ConformalIntervals_empirical,naive_mean_ConformalIntervals_empirical_residual,naive_mean_Fallback,naive_mean_NaiveVariance,naive_mean_SquaringResiduals
runtime_secs,6.362267,6.245747,6.282353,6.651181,9.395198,10.006466,9.900707,9.373487,1.160173,96.805224,...,2.830332,2.786863,2.711761,2.133101,2.039982,2.158137,2.026803,0.81366,15.317874,75.839118
CRPS_fold_0_test,60.371438,70.170681,51.709226,50.510945,335.259578,110.494639,107.914892,72.406138,105.09359,112.443789,...,56.809502,78.565536,66.930454,290.660829,93.071283,88.783517,56.182452,94.285897,95.146744,64.045408
CRPS_fold_1_test,93.681197,231.323649,182.873104,151.469943,314.317786,117.744539,121.240994,101.557175,127.059052,129.250736,...,97.255432,107.124823,97.306225,304.704553,119.007448,118.492787,100.331681,126.579129,126.90462,107.922859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IntervalWidth_fold_33_test,157.136271,46.269592,51.121199,46.834957,755.850306,509.821237,484.617664,17.426425,566.315105,542.2315,...,19.219726,0.002198,22.211057,753.939136,442.855648,494.307276,2.434177,529.380571,513.969055,323.349303
IntervalWidth_fold_34_test,71.493966,40.249536,41.776832,54.474649,538.725141,365.951068,385.326239,11.032149,445.09094,452.529597,...,22.41989,0.004087,16.926494,670.913866,378.318975,371.830469,0.003506,417.924185,407.280499,74.488371
IntervalWidth_fold_35_test,86.20565,23.10723,31.004127,32.386724,429.272931,308.407725,291.145279,6.218093,296.633081,287.387068,...,5.249246,0.014323,6.357369,468.591502,299.370991,247.036911,0.0,277.839558,277.322186,260.848862
IntervalWidth_mean,144.659816,118.631912,135.133736,144.765565,1098.020116,709.457976,687.622588,37.829966,751.24693,756.067716,...,23.721323,0.023886,26.744834,1060.94109,674.602262,648.129107,31.331248,695.703221,698.249792,522.260465


## Creating rank based evaluation
To enable better interpretation of the results, we calculated the average ranks, that are also reported in the paper.

In [13]:
pd.options.mode.copy_on_write = True
pd.set_option("display.precision", 2)

results_df = pd.read_csv("australian_electricity.csv")
results_df[results_df.model_id.apply(lambda x: not x.startswith("StatsForecastAutoARIMA"))]
dfs = []

for ds_id in ["T1", "T2", "T3", "T4", "T5"]:
    show_df = results_df[["model_id","validation_id", "CRPS_mean", "PinballLoss_mean", "runtime_secs", "AUCalibration_mean"]]
    show_df["ds_id"] = show_df["validation_id"].apply(lambda x: get_ds_id(x))
    if ds_id == "T1":
        # Unfortunately renamed Bonferroni once to M
        show_df = show_df[show_df["model_id"].map(lambda x: not x.endswith("conformal_bonferroni"))]

    show_df["wrapper_id"] = show_df["model_id"].apply(lambda x: get_wrapper_id(x))
    show_df["model_id"] = show_df["model_id"].apply(lambda x: get_model_name(x))
    show_df = show_df[show_df["ds_id"] == ds_id]
    
    show_df = show_df.set_index(["wrapper_id", "model_id"])

    show_df = show_df[[ "CRPS_mean", "PinballLoss_mean", "AUCalibration_mean", "runtime_secs", ]]
    show_df = show_df.groupby("model_id").rank().groupby("wrapper_id").mean()
    dfs.append(show_df)
df = pd.concat(dfs, keys=["T1", "T2", "T3", "T4", "T5"], names=["Dataset"]).groupby("wrapper_id").mean()
df = df.loc[["Fallback", "Naive Variance", "Squaring Residuals","BlockDistributionBootstrap", 'BlockResidualBootstrap',
    'BlockStatisticPreservingBootstrap', 'MovingBlockBootstrap', 'CI Conformal',
    'CI Empirical', 'CI Empirical Residual' , 'CI Bonferroni ', ]]
df.to_latex("elec.tex", float_format="%.2f")
df

,CRPS_mean,PinballLoss_mean,AUCalibration_mean,runtime_secs
wrapper_id,,,,
BlockDistributionBootstrap,2.50,2.50,7.00,8.00
BlockResidualBootstrap,3.50,4.50,7.00,8.50
BlockStatisticPreservingBootstrap,4.00,5.75,4.75,7.25
CI Bonferroni,10.50,8.25,5.50,3.25
CI Conformal,7.00,6.00,5.00,3.75
CI Empirical,4.25,3.25,8.50,3.25
CI Empirical Residual,3.00,5.00,6.50,3.75
Fallback,9.50,9.75,5.88,1.00
MovingBlockBootstrap,5.25,6.50,3.00,6.25


In [11]:
pd.options.mode.copy_on_write = True
pd.set_option("display.precision", 2)

results_df = pd.read_csv("sunspot.csv")
results_df[results_df.model_id.apply(lambda x: not x.startswith("StatsForecastAutoARIMA"))]
dfs = []

show_df = results_df[["model_id","validation_id", "CRPS_mean", "PinballLoss_mean", "runtime_secs", "AUCalibration_mean"]]
show_df["wrapper_id"] = show_df["model_id"].apply(lambda x: get_wrapper_id(x))
show_df["model_id"] = show_df["model_id"].apply(lambda x: get_model_name(x))
show_df = show_df.set_index(["wrapper_id", "model_id"])
show_df = show_df[[ "CRPS_mean", "PinballLoss_mean", "AUCalibration_mean", "runtime_secs", ]]
show_df = show_df.groupby("model_id").rank().groupby("wrapper_id").mean()

df = show_df.loc[["Fallback", "Naive Variance", "Squaring Residuals","BlockDistributionBootstrap", 'BlockResidualBootstrap',
    'BlockStatisticPreservingBootstrap', 'MovingBlockBootstrap', 'CI Conformal',
    'CI Empirical', 'CI Empirical Residual' , 'CI Bonferroni ', ]]
df.to_latex("sunspot.tex", float_format="%.2f")
df

,CRPS_mean,PinballLoss_mean,AUCalibration_mean,runtime_secs
wrapper_id,,,,
Fallback,9.00,7.25,9.50,1.00
Naive Variance,6.50,6.25,7.75,10.00
Squaring Residuals,8.00,8.00,4.75,11.00
BlockDistributionBootstrap,1.00,1.00,3.00,7.25
BlockResidualBootstrap,6.00,6.25,5.00,6.75
BlockStatisticPreservingBootstrap,5.50,7.00,2.50,6.25
MovingBlockBootstrap,5.75,5.75,7.50,5.00
CI Conformal,4.75,4.50,6.75,4.50
CI Empirical,5.50,4.00,7.25,4.50


In [12]:
pd.options.mode.copy_on_write = True
pd.set_option("display.precision", 2)

results_df = pd.read_csv("us_birth.csv")
results_df[results_df.model_id.apply(lambda x: not x.startswith("StatsForecastAutoARIMA"))]
dfs = []

show_df = results_df[["model_id","validation_id", "CRPS_mean", "PinballLoss_mean", "runtime_secs", "AUCalibration_mean"]]
show_df["wrapper_id"] = show_df["model_id"].apply(lambda x: get_wrapper_id(x))
show_df["model_id"] = show_df["model_id"].apply(lambda x: get_model_name(x))
show_df = show_df.set_index(["wrapper_id", "model_id"])

show_df = show_df[[ "CRPS_mean", "PinballLoss_mean", "AUCalibration_mean", "runtime_secs", ]]
show_df = show_df.groupby("model_id").rank().groupby("wrapper_id").mean()

df = show_df.loc[["Fallback", "Naive Variance", "Squaring Residuals","BlockDistributionBootstrap", 'BlockResidualBootstrap',
    'BlockStatisticPreservingBootstrap', 'MovingBlockBootstrap', 'CI Conformal',
    'CI Empirical', 'CI Empirical Residual' , 'CI Bonferroni ', ]]
df.to_latex("usbirth.tex", float_format="%.2f")
df

,CRPS_mean,PinballLoss_mean,AUCalibration_mean,runtime_secs
wrapper_id,,,,
Fallback,9.25,9.50,7.88,1.00
Naive Variance,6.25,5.75,6.25,10.00
Squaring Residuals,9.50,8.50,6.50,11.00
BlockDistributionBootstrap,1.00,1.00,8.25,7.25
BlockResidualBootstrap,3.75,5.75,5.50,7.00
BlockStatisticPreservingBootstrap,5.50,6.00,7.38,6.25
MovingBlockBootstrap,5.75,5.25,5.12,4.75
CI Conformal,6.00,6.00,4.75,4.25
CI Empirical,5.50,4.50,4.88,4.75
